# Image viewer

In [2]:
%matplotlib widget
DEBUG = False

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from glob import glob
from skimage.io import imread
from skimage.color import rgb2gray
from ipyfilechooser import FileChooser
import traceback
import functools
import fnmatch
from astropy.io import fits
from scipy.ndimage import median_filter

FILE_PATTERNS = ['*.png', '*.jpg', '*.npy', '*.fits']

# Set the filter pattern to only allow PNG and JPG files
folder_selector = FileChooser(
    os.path.realpath('.'),
    show_only_dirs=True,
    select_desc="Select folder",
    change_desc="Change folder",
)

folder_selector.layout = widgets.Layout(width='600px')  # Expand width of the folder selection
folder_selector._select.layout = widgets.Layout(width="200px")

file_selection_widget = widgets.Select(
    options=[],
    description="Files in folder:",
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)
refresh_button = widgets.Button(description="Refresh file list", style={'description_width': 'initial'})
q99_button = widgets.Button(description="99%", style={'description_width': 'initial'})

normalize_to_max_checkbox = widgets.Checkbox(value=False, description='Normalize to max')
apply_median_filter_checkbox = widgets.Checkbox(value=False, description='Apply median 3x3 filter')

vmin_vmax_slider = widgets.FloatRangeSlider(value=[0, 1], min=0, max=1., step=0.001, description='Contrast:', layout=widgets.Layout(width=f'{7*72}px'))
logy_checkbox = widgets.Checkbox(value=False, description='Log y scale', layout=widgets.Layout(width=f'{3*72}px'))
x_range_slider = widgets.IntRangeSlider(value=[0, 1], min=0, max=1, step=1, description='X Range:', layout=widgets.Layout(width=f'{8.5*72}px'))
y_range_slider = widgets.IntRangeSlider(value=[0, 1], min=0, max=1, step=1, description='Y Range:', orientation='vertical', layout=widgets.Layout(height=f'{7*72}px'))
error_widget = widgets.HTML() #widgets.Label() #widgets.HTML()

#fig, axes = plt.subplots(2, 3, figsize=(12, 8), gridspec_kw={'width_ratios': [4, 1, 1], 'height_ratios': [4, 1]})
#ax_img, ax_hist, ax_right = axes[0]
#ax_bottom, _, _ = axes[1]

plt.ioff() # Avoid displaying automatically - will need to be re-enabled at the end
fig_contrast, ax_hist = plt.subplots(1, 1, figsize=(6.5, 0.5))
fig_contrast.subplots_adjust(left=0, right=1, top=1, bottom=0)

fig_colorbar, ax_colorbar = plt.subplots(1, 1, figsize=(6.5, 0.25))
fig_colorbar.subplots_adjust(left=0, right=1, top=1, bottom=0)

fig, axes = plt.subplots(2, 2, figsize=(6, 6), gridspec_kw={'width_ratios': [4, 1], 'height_ratios': [4, 1], })
ax_img, ax_right = axes[0]
ax_bottom, ax_empty = axes[1]
fig.subplots_adjust(left=0.1, right=1, top=1, bottom=0.05)

#ax_img.set_title("Image")
#ax_bottom.set_title("Integrated X")
#ax_right.set_title("Integrated Y")
ax_empty.clear()  # Remove any existing content
ax_empty.set_xticks([])  # Remove x-axis ticks
ax_empty.set_yticks([])  # Remove y-axis ticks
ax_empty.set_xticklabels([])  # Remove x-axis tick labels
ax_empty.set_yticklabels([])  # Remove y-axis tick labels
ax_empty.set_frame_on(False)  # Remove the frame

img = None
original_image = None

def filter_files(filenames, patterns):
    return [f for f in filenames if any(fnmatch.fnmatch(f, p) for p in patterns)]

def format_exception_html(exception):
    """Formats an exception as an HTML string with styling. UNSAFE!"""
    formatted_traceback = traceback.format_exc()
    return f"""
    <div style="color: red; font-family: monospace; white-space: pre-wrap;">
        <b>Error:</b> {str(exception)}<br>
        <pre>{formatted_traceback}</pre>
    </div>
    """

def catch_errors(widget):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            widget.value = ""
            try:
                return func(*args, **kwargs)
            except Exception as e:
                #error_message = f'```Error: SOME ERROR\nLine 1\n\nLine 2\n\n Line 3\n\n```\n</span>'
                #error_message = f'```\nError: {str(e)}\n{traceback.format_exc()}\n```\n</span>'
                #html = markdown.markdown(error_message)
                #widget.value = f'<span style="color: #ff0000;">{html}</span>'  # Update the widget with the error message
                widget.value = format_exception_html(e)
        return wrapper
    return decorator

def update_files_in_folder(change=None):
    folder = folder_selector.selected
    files = os.listdir(folder)
    # Only files matching, and sort alphabetically
    filtered_files = sorted(filter_files(files, FILE_PATTERNS))

    # Update widget
    file_selection_widget.options = filtered_files
    if filtered_files:
        # Select the first file
        file_selection_widget.value = file_selection_widget.options[0]

def update_hist_yscale(change=None):
    global ax_hist, ax_colorbar, logy_checkbox

    for ax in [ax_colorbar, ax_hist]:
        if logy_checkbox.value:
            ax.set_yscale('log')
            ax.relim() # recompute the data limits
            ax.autoscale(axis='y') 
            ax.set_ylim(0.1, None)
        else:
            ax.set_yscale('linear')
            ax.relim() # recompute the data limits
            ax.autoscale(axis='y') 
            ax.set_ylim(0, None)
        

@catch_errors(error_widget)
def reload_image(change=None):
    global img
    global original_image
    
    folder = folder_selector.selected
    # If both are not None
    if folder and file_selection_widget.value:
        file_path = os.path.join(folder, file_selection_widget.value)
    else:
        file_path = None
    # Returns if nothing is selected (but first clear all)
    if not file_path or not os.path.exists(file_path):
        ax_hist.clear()
        ax_colorbar.clear()
        ax_img.clear()
        ax_right.clear()
        ax_bottom.clear()
        return
    
    # Load and preprocess image
    if file_path.endswith('.npy'):
        img = np.load(file_path)
        # Image needs to be flipped upside-down
        img = np.flipud(img)
    elif file_path.endswith('.fits'):
        with fits.open(file_path) as hdul:
            hdu = hdul[0]
            img = hdu.data.copy()
    else:
        img = imread(file_path)
        # Image needs to be flipped upside-down
        img = np.flipud(img)
    
    if len(img.shape) == 3 and img.shape[2] == 4: # RGBA, convert to RGB on white background
        alpha_channel = img[:,:,3]
        rgb_channels = img[:,:,:3]

        # White Background Image
        white_background_image = np.ones_like(rgb_channels, dtype=np.uint8) * 255

        # Alpha factor
        alpha_factor = alpha_channel[:,:,np.newaxis].astype(np.float32) / 255
        alpha_factor = np.concatenate((alpha_factor,alpha_factor,alpha_factor), axis=2)
        
        # Transparent Image Rendered on White Background
        base = rgb_channels.astype(np.float32) * alpha_factor
        white = white_background_image.astype(np.float32) * (1 - alpha_factor)
        final_image = base + white
        original_image = img
        img = final_image.astype(np.uint8)
        #img = rgb_channels

    # Convert to RGB
    if len(img.shape) == 3:
        img = rgb2gray(img)    

    if DEBUG:
        error_widget.value = f"BEFORE: {img.dtype=}, {img.min()=}, {img.max()=}<br>"
    if img.dtype in (np.uint8, np.uint16):
        img = img.astype(float) / np.iinfo(img.dtype).max    
    if apply_median_filter_checkbox.value:
        img = median_filter(img, size=3)
#        if DEBUG:
#            error_widget.value += f"AFTER FILTER: {img.dtype=}, {img.min()=}, {img.max()=}<br>"
    if normalize_to_max_checkbox.value:
        img /= img.max()
    if DEBUG:
        error_widget.value += f"AFTER: {img.dtype=}, {img.min()=}, {img.max()=}<br>"

    # Update sliders based on image dimensions
    x_range_slider.min, x_range_slider.max = 0, img.shape[1]
    y_range_slider.min, y_range_slider.max = 0, img.shape[0]
    x_range_slider.value = 0, img.shape[1]
    y_range_slider.value = 0, img.shape[0]

    update_image(change=change)

def update_image(change=None):
    global img, ax_img, ax_hist, ax_bottom, ax_right

    # Update histogram
    hist, bins = np.histogram(img.flatten(), bins=256, range=(0, 1))
    for ax in [ax_colorbar, ax_hist]:
        ax.clear()
        ax.plot((bins[:-1] + bins[1:])/2, hist, color='black')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_xticks([])
        ax.set_yticks([])
        update_hist_yscale() # Calling it again as it is reset

    # x range
    ax_hist.set_xlim(0, 1)
    vmin, vmax = vmin_vmax_slider.value

    for ax in [ax_colorbar, ax_hist]:
        y_min, y_max = ax.get_ylim()
        gradient = np.linspace(0, 1, 256).reshape(1, -1)  # Horizontal gradient
        ax.imshow(gradient, extent=[vmin, vmax, y_min, y_max], aspect='auto', cmap='gray', alpha=0.5)

    ax_colorbar.set_xlim(vmin, vmax)
    ax_hist.axvline(vmin, color='red', linestyle='--')
    ax_hist.axvline(vmax, color='blue', linestyle='--')
    
    # Update image display
    ax_img.clear()    
    ax_img.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    ax_img.set_xlim(0, img.shape[1])
    ax_img.set_ylim(0, img.shape[0])
    ax_img.set_xticklabels([])
    ax_img.set_yticklabels([])
    ax_img.set_xticks([])
    ax_img.set_yticks([])

    
    # Update crop lines
    x1, x2 = x_range_slider.value
    y1, y2 = y_range_slider.value
    ax_img.axvline(x1, color='red', linestyle='--')
    ax_img.axvline(x2, color='red', linestyle='--')
    ax_img.axhline(y1, color='blue', linestyle='--')
    ax_img.axhline(y2, color='blue', linestyle='--')
    
    # Extract cropped region and update projections
    cropped = img[int(y1):int(y2), int(x1):int(x2)]
    
    ax_bottom.clear()
    ax_bottom.plot(np.mean(cropped, axis=0))
    ax_bottom.set_xlim(0, cropped.shape[1])
    
    ax_right.clear()
    ax_right.plot(np.mean(cropped, axis=1), np.arange(len(np.mean(cropped, axis=1))))
    ax_right.set_ylim(0, cropped.shape[0])


def qunatile_histogram(data, q=0.95, bins=100) :
    lower = (1 - q) / 2
    upper = 1 - lower
    low_val, high_val = np.quantile(data, [lower, upper])

    # Filter data within quantile range
    filtered_data = data[(data >= low_val) & (data <= high_val)]

    # Create histogram
    hist, bin_edges = np.histogram(filtered_data, bins=256)
    
    return history,bin_edges

def set_quantile_interval(change=None):
    q=0.99
    
    lower = (1 - q) / 2
    upper = 1 - lower
    
    low_val, high_val = np.quantile(img.ravel(), [lower, upper])
    print(low_val,high_val)
    vmin_vmax_slider.value=(low_val,high_val)
    
# Link events
folder_selector.register_callback(update_files_in_folder)
file_selection_widget.observe(reload_image, names='value')
vmin_vmax_slider.observe(update_image, names='value')
x_range_slider.observe(update_image, names='value')
y_range_slider.observe(update_image, names='value')
refresh_button.on_click(update_files_in_folder)
q99_button.on_click(set_quantile_interval)
logy_checkbox.observe(update_hist_yscale, names='value')
apply_median_filter_checkbox.observe(reload_image, names='value')
normalize_to_max_checkbox.observe(reload_image, names='value')

display(widgets.VBox([folder_selector,
                      widgets.HBox([file_selection_widget, refresh_button]),
                      error_widget,
                      widgets.HBox([normalize_to_max_checkbox, apply_median_filter_checkbox]),
                      fig_contrast.canvas,
                      widgets.HBox([q99_button,logy_checkbox, vmin_vmax_slider]), 
                      fig_colorbar.canvas,
                      widgets.HBox([y_range_slider,fig.canvas]),
                      x_range_slider, 
                      ]))

# Reactivate interactivity now that everything is displayed
plt.ion()
# Initialize with first image
reload_image()

In [3]:
file_selection_widget.value

In [9]:
import re
def make_file_mask(filename) :
    match = re.search(r'_(0*\d+)(\.fits)$', filename)
    if match:
        original_number = match.group(1)  # Extract the original number with padding
        num_digits = len(original_number)  # Determine the length of the original padding
        return re.sub(r'_(0*\d+)(\.fits)$', f'_{{0:0{num_digits}d}}\\2', filename)
    return filename  # Return unchanged if no match found

In [10]:
names = ['cal_00001.fits','var_cal_0001.fits','c_v_cal_01.fits','cal_1.fits']
for name in names : 
    print(name,make_file_mask(name))

cal_00001.fits cal_{0:05d}.fits
var_cal_0001.fits var_cal_{0:04d}.fits
c_v_cal_01.fits c_v_cal_{0:02d}.fits
cal_1.fits cal_{0:01d}.fits


In [11]:
import os

def list_matching_files(directory, pattern="file_*.ext"):
    return sorted(glob(f"{directory}/{pattern}"))

def find_first_last_indices(filenames, pattern=r'mask_(\d{4})\.fits'):
    indices = []

    
    for filename in filenames:
        filename = filename.split('/')[-1]
        match=re.search(r'_(0*\d+)(\.fits)$', filename)
        if match:
            indices.append(int(match.group(1)))  # Convert to integer

    if indices:
        return min(indices), max(indices)
    else:
        return None, None  # If no matches found

In [12]:
flist=list_matching_files('/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/',r'*.fits')

In [13]:
find_first_last_indices(flist,'cal_{0:05d}.fits')

(1, 125)

In [9]:
flist

['/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00001.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00002.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00003.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00004.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00005.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00006.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00007.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00008.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00009.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00010.fits',
 '/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_0

In [11]:
str='/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/00_Calibration/cal_00001.fits'
s=str.split('/')[-1]

In [13]:
ext = s.split('.')[-1]
print(s,ext)

cal_00001.fits fits


# Image browser

In [14]:
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from glob import glob
from skimage.io import imread
from skimage.color import rgb2gray
from ipyfilechooser import FileChooser
import traceback
import functools
import fnmatch
from astropy.io import fits
from scipy.ndimage import median_filter

FILE_PATTERNS = ['*.png', '*.jpg', '*.npy', '*.fits']

# Set the filter pattern to only allow PNG and JPG files
folder_selector = FileChooser(
    os.path.realpath('.'),
    show_only_dirs=True,
    select_desc="Select folder",
    change_desc="Change folder",
)

folder_selector.layout = widgets.Layout(width='600px')  # Expand width of the folder selection
folder_selector._select.layout = widgets.Layout(width="200px")

file_selection_widget = widgets.Select(
    options=[],
    description="Files in folder:",
    style={'description_width': 'initial'},
    layout={'width': '400px'}
)
refresh_button = widgets.Button(description="Refresh file list", style={'description_width': 'initial'})
q99_button = widgets.Button(description="99%", style={'description_width': 'initial'})

normalize_to_max_checkbox = widgets.Checkbox(value=False, description='Normalize to max')
apply_median_filter_checkbox = widgets.Checkbox(value=False, description='Apply median 3x3 filter')

vmin_vmax_slider = widgets.FloatRangeSlider(value=[0, 1], min=0, max=1., step=0.001, description='Contrast:', layout=widgets.Layout(width=f'{7*72}px'))
# logy_checkbox = widgets.Checkbox(value=False, description='Log y scale', layout=widgets.Layout(width=f'{3*72}px'))
# x_range_slider = widgets.IntRangeSlider(value=[0, 1], min=0, max=1, step=1, description='X Range:', layout=widgets.Layout(width=f'{8.5*72}px'))
# y_range_slider = widgets.IntRangeSlider(value=[0, 1], min=0, max=1, step=1, description='Y Range:', orientation='vertical', layout=widgets.Layout(height=f'{7*72}px'))
error_widget = widgets.HTML() #widgets.Label() #widgets.HTML()


plt.ioff() # Avoid displaying automatically - will need to be re-enabled at the end
fig_contrast, ax_hist = plt.subplots(1, 1, figsize=(6.5, 0.5))
fig_contrast.subplots_adjust(left=0, right=1, top=1, bottom=0)

# fig_colorbar, ax_colorbar = plt.subplots(1, 1, figsize=(6.5, 0.25))
# fig_colorbar.subplots_adjust(left=0, right=1, top=1, bottom=0)

fig, axes = plt.subplots(2, 2, figsize=(6, 6), gridspec_kw={'width_ratios': [4, 1], 'height_ratios': [4, 1], })
ax_img, ax_right = axes[0]
ax_bottom, ax_empty = axes[1]
fig.subplots_adjust(left=0.1, right=1, top=1, bottom=0.05)

#ax_img.set_title("Image")
#ax_bottom.set_title("Integrated X")
#ax_right.set_title("Integrated Y")
ax_empty.clear()  # Remove any existing content
ax_empty.set_xticks([])  # Remove x-axis ticks
ax_empty.set_yticks([])  # Remove y-axis ticks
ax_empty.set_xticklabels([])  # Remove x-axis tick labels
ax_empty.set_yticklabels([])  # Remove y-axis tick labels
ax_empty.set_frame_on(False)  # Remove the frame

img = None
original_image = None

def filter_files(filenames, patterns):
    return [f for f in filenames if any(fnmatch.fnmatch(f, p) for p in patterns)]

def format_exception_html(exception):
    """Formats an exception as an HTML string with styling. UNSAFE!"""
    formatted_traceback = traceback.format_exc()
    return f"""
    <div style="color: red; font-family: monospace; white-space: pre-wrap;">
        <b>Error:</b> {str(exception)}<br>
        <pre>{formatted_traceback}</pre>
    </div>
    """

def catch_errors(widget):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            widget.value = ""
            try:
                return func(*args, **kwargs)
            except Exception as e:
                #error_message = f'```Error: SOME ERROR\nLine 1\n\nLine 2\n\n Line 3\n\n```\n</span>'
                #error_message = f'```\nError: {str(e)}\n{traceback.format_exc()}\n```\n</span>'
                #html = markdown.markdown(error_message)
                #widget.value = f'<span style="color: #ff0000;">{html}</span>'  # Update the widget with the error message
                widget.value = format_exception_html(e)
        return wrapper
    return decorator

def update_files_in_folder(change=None):
    folder = folder_selector.selected
    files = os.listdir(folder)
    # Only files matching, and sort alphabetically
    filtered_files = sorted(filter_files(files, FILE_PATTERNS))

    # Update widget
    file_selection_widget.options = filtered_files
    if filtered_files:
        # Select the first file
        file_selection_widget.value = file_selection_widget.options[0]

def update_hist_yscale(change=None):
    global ax_hist, ax_colorbar, logy_checkbox

    for ax in [ax_colorbar, ax_hist]:
        if logy_checkbox.value:
            ax.set_yscale('log')
            ax.relim() # recompute the data limits
            ax.autoscale(axis='y') 
            ax.set_ylim(0.1, None)
        else:
            ax.set_yscale('linear')
            ax.relim() # recompute the data limits
            ax.autoscale(axis='y') 
            ax.set_ylim(0, None)
        

@catch_errors(error_widget)
def reload_image(change=None):
    global img
    global original_image
    
    folder = folder_selector.selected
    # If both are not None
    if folder and file_selection_widget.value:
        file_path = os.path.join(folder, file_selection_widget.value)
    else:
        file_path = None
    # Returns if nothing is selected (but first clear all)
    if not file_path or not os.path.exists(file_path):
        ax_hist.clear()
        ax_colorbar.clear()
        ax_img.clear()
        ax_right.clear()
        ax_bottom.clear()
        return
    
    # Load and preprocess image
    if file_path.endswith('.npy'):
        img = np.load(file_path)
        # Image needs to be flipped upside-down
        img = np.flipud(img)
    elif file_path.endswith('.fits'):
        with fits.open(file_path) as hdul:
            hdu = hdul[0]
            img = hdu.data.copy()
    else:
        img = imread(file_path)
        # Image needs to be flipped upside-down
        img = np.flipud(img)
    
    if len(img.shape) == 3 and img.shape[2] == 4: # RGBA, convert to RGB on white background
        alpha_channel = img[:,:,3]
        rgb_channels = img[:,:,:3]

        # White Background Image
        white_background_image = np.ones_like(rgb_channels, dtype=np.uint8) * 255

        # Alpha factor
        alpha_factor = alpha_channel[:,:,np.newaxis].astype(np.float32) / 255
        alpha_factor = np.concatenate((alpha_factor,alpha_factor,alpha_factor), axis=2)
        
        # Transparent Image Rendered on White Background
        base = rgb_channels.astype(np.float32) * alpha_factor
        white = white_background_image.astype(np.float32) * (1 - alpha_factor)
        final_image = base + white
        original_image = img
        img = final_image.astype(np.uint8)
        #img = rgb_channels

    # Convert to RGB
    if len(img.shape) == 3:
        img = rgb2gray(img)    

    if DEBUG:
        error_widget.value = f"BEFORE: {img.dtype=}, {img.min()=}, {img.max()=}<br>"
    if img.dtype in (np.uint8, np.uint16):
        img = img.astype(float) / np.iinfo(img.dtype).max    
    if apply_median_filter_checkbox.value:
        img = median_filter(img, size=3)
#        if DEBUG:
#            error_widget.value += f"AFTER FILTER: {img.dtype=}, {img.min()=}, {img.max()=}<br>"
    if normalize_to_max_checkbox.value:
        img /= img.max()
    if DEBUG:
        error_widget.value += f"AFTER: {img.dtype=}, {img.min()=}, {img.max()=}<br>"

    # Update sliders based on image dimensions
    x_range_slider.min, x_range_slider.max = 0, img.shape[1]
    y_range_slider.min, y_range_slider.max = 0, img.shape[0]
    x_range_slider.value = 0, img.shape[1]
    y_range_slider.value = 0, img.shape[0]

    update_image(change=change)

def update_image(change=None):
    global img, ax_img, ax_hist, ax_bottom, ax_right

    # Update histogram
    hist, bins = np.histogram(img.flatten(), bins=256, range=(0, 1))
    for ax in [ax_colorbar, ax_hist]:
        ax.clear()
        ax.plot((bins[:-1] + bins[1:])/2, hist, color='black')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_xticks([])
        ax.set_yticks([])
        update_hist_yscale() # Calling it again as it is reset

    # x range
    ax_hist.set_xlim(0, 1)
    vmin, vmax = vmin_vmax_slider.value

    for ax in [ax_colorbar, ax_hist]:
        y_min, y_max = ax.get_ylim()
        gradient = np.linspace(0, 1, 256).reshape(1, -1)  # Horizontal gradient
        ax.imshow(gradient, extent=[vmin, vmax, y_min, y_max], aspect='auto', cmap='gray', alpha=0.5)

    ax_colorbar.set_xlim(vmin, vmax)
    ax_hist.axvline(vmin, color='red', linestyle='--')
    ax_hist.axvline(vmax, color='blue', linestyle='--')
    
    # Update image display
    ax_img.clear()    
    ax_img.imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    ax_img.set_xlim(0, img.shape[1])
    ax_img.set_ylim(0, img.shape[0])
    ax_img.set_xticklabels([])
    ax_img.set_yticklabels([])
    ax_img.set_xticks([])
    ax_img.set_yticks([])

    
    # Update crop lines
    x1, x2 = x_range_slider.value
    y1, y2 = y_range_slider.value
    ax_img.axvline(x1, color='red', linestyle='--')
    ax_img.axvline(x2, color='red', linestyle='--')
    ax_img.axhline(y1, color='blue', linestyle='--')
    ax_img.axhline(y2, color='blue', linestyle='--')
    
    # Extract cropped region and update projections
    cropped = img[int(y1):int(y2), int(x1):int(x2)]
    
    ax_bottom.clear()
    ax_bottom.plot(np.mean(cropped, axis=0))
    ax_bottom.set_xlim(0, cropped.shape[1])
    
    ax_right.clear()
    ax_right.plot(np.mean(cropped, axis=1), np.arange(len(np.mean(cropped, axis=1))))
    ax_right.set_ylim(0, cropped.shape[0])

def set_quantile_interval(change=None):
    q=0.99
    rq2=(1-q)/2.0
    
    hist, bins = np.histogram(img.flatten(), bins=256, range=(0, 1))
    chist = np.cumsum(hist)
    chist = chist/chist.max()
    
    start_idx = np.searchsorted(chist, rq2, side='left')
    end_idx = np.searchsorted(chist, q+rq2, side='right')
    
    vmin_vmax_slider.value=(bins[start_idx],bins[end_idx])
# Link events
folder_selector.register_callback(update_files_in_folder)
file_selection_widget.observe(reload_image, names='value')
vmin_vmax_slider.observe(update_image, names='value')
x_range_slider.observe(update_image, names='value')
y_range_slider.observe(update_image, names='value')
refresh_button.on_click(update_files_in_folder)
q99_button.on_click(set_quantile_interval)
logy_checkbox.observe(update_hist_yscale, names='value')
apply_median_filter_checkbox.observe(reload_image, names='value')
normalize_to_max_checkbox.observe(reload_image, names='value')

display(widgets.VBox([folder_selector,
                      widgets.HBox([file_selection_widget, refresh_button]),
                      error_widget,
                      widgets.HBox([normalize_to_max_checkbox, apply_median_filter_checkbox]),
                      fig_contrast.canvas,
                      widgets.HBox([q99_button,logy_checkbox, vmin_vmax_slider]), 
                      fig_colorbar.canvas,
                      widgets.HBox([y_range_slider,fig.canvas]),
                      x_range_slider, 
                      ]))

# Reactivate interactivity now that everything is displayed
plt.ion()
# Initialize with first image
reload_image()

# File selection widget

In [15]:
FILE_PATTERNS = ['*.png', '*.jpg', '*.npy', '*.fits']

# Set the filter pattern to only allow PNG and JPG files
folder_selector = FileChooser(
    os.path.realpath('.'),
    show_only_dirs=False,
    select_desc="Browse file",
    change_desc="Change file",
)

folder_selector.layout = widgets.Layout(width='800px')  # Expand width of the folder selection
folder_selector._select.layout = widgets.Layout(width="200px")

In [16]:
display(folder_selector)

FileChooser(path='/Users/kaestner/git/scripts/python/imagingUI', filename='', title='', show_hidden=False, sel…

In [17]:
folder_selector.selected

'/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/TS_00065.fits'

# File mask extractor

In [36]:
import os
from ipyfilechooser import FileChooser
import ipywidgets as widgets
import sys
sys.path.append('../')
import amglib.readers as rd

class FileSelector:
    FILE_PATTERNS = ['*.png', '*.jpg', '*.npy', '*.fits', '*.tif', '*.tiff']
    
    @property
    def file_name(self) :
        return self._fileinfo["name"]
    
    @property 
    def file_path(self) :
        return self._fileinfo["path"]
    
    @property 
    def file_mask(self) :
        return self._fileinfo["mask"]
    
    @property 
    def first_index(self) :
        return self._fileinfo["first"]
    
    @property
    def last_index(self) :
        return self._fileinfo["last"]
    
    @property
    def info(self) :
        return self._fileinfo

    def __init__(self, start_dir='.'):
        self.file_chooser = FileChooser(
            os.path.realpath(start_dir),
            show_only_dirs=False,
            select_desc="Browse file",
            change_desc="Change file",
#             filter_pattern=";".join(self.FILE_PATTERNS)
        )
        self.file_chooser.layout = widgets.Layout(width='800px')
        self.file_chooser._select.layout = widgets.Layout(width="200px")
        self.file_chooser.register_callback(self.on_file_selected)

    def on_file_selected(self, chooser):
        selected = chooser.selected
        if selected:
            self.analyze_file(selected)

    def analyze_file(self, filename):
        # Implement your analysis logic here
        fname      = self.file_chooser.selected
        path       = os.path.dirname(fname)
        ext        = fname.split('.')[-1]
        flist      = list_matching_files(path,r'_'.join(fname.split('/')[-1].split('_')[:-1])+'*.'+ext)
        fmask      = rd.make_file_mask(fname)
        first,last = rd.find_first_last_indices(flist,fmask.split('/')[-1])

        self._fileinfo = { "name" : fname,
                           "mask" : fmask,
                           "path" : path,
                           "ext"  : ext,
                           "first": first,
                           "last" : last,
                           "files": flist}

    def display(self):
        display(self.file_chooser)

# Usage:
analyzer = FileSelector()
analyzer.display()

FileChooser(path='/Users/kaestner/git/scripts/python/imagingUI', filename='', title='', show_hidden=False, sel…

In [25]:
analyzer.file_mask

'/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_{0:05d}.fits'

In [37]:
analyzer.info

{'name': '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_00001.fits',
 'mask': '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_{0:05d}.fits',
 'path': '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1',
 'ext': 'fits',
 'first': 1,
 'last': 5,
 'files': ['/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_00001.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_00002.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_00003.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_00004.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_00005.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_2_00001.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_2_00002.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/

In [34]:
s = '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/BBOB_00002.fits'
print('_'.join(s.split('/')[-1].split('_')[:-1]))

BBOB


In [39]:
import amglib.widgets as aw
fs = aw.FileSelector()
fs.display()

FileChooser(path='/Users/kaestner/git/scripts/python/imagingUI', filename='', title='', show_hidden=False, sel…

FileChooser(path='/Users/kaestner/git/scripts/python/imagingUI', filename='', title='', show_hidden=False, sel…

In [41]:
import importlib
importlib.reload(aw)
fs = aw.FileSelector()
fs.display()

FileChooser(path='/Users/kaestner/git/scripts/python/imagingUI', filename='', title='', show_hidden=False, sel…

FileChooser(path='/Users/kaestner/git/scripts/python/imagingUI', filename='', title='', show_hidden=False, sel…

In [43]:
fs.info

{'name': '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/DC_00001.fits',
 'mask': '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/DC_{0:05d}.fits',
 'path': '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1',
 'ext': 'fits',
 'first': 1,
 'last': 5,
 'files': ['/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/DC_00001.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/DC_00002.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/DC_00003.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/DC_00004.fits',
  '/Users/Shared/Data/P20240876/02_rawdata/04_Straw_NR_TR_yes_T27_R1/DC_00005.fits']}